In [ ]:
#First Lets Create the entropy and apply one hot encoding to all categories


In [32]:
#import block

import pandas as pd
import category_encoders as ce
import numpy as np
from scipy.stats import entropy


In [57]:
tab_data_unfiltered = pd.read_csv("data\motionStrike_TVcodes_data.tsv", delimiter = '\t')
tab_data_unfiltered

<>:1: SyntaxWarning: invalid escape sequence '\m'
<>:1: SyntaxWarning: invalid escape sequence '\m'
C:\Users\aarya\AppData\Local\Temp\ipykernel_3976\680595952.py:1: SyntaxWarning: invalid escape sequence '\m'
  tab_data_unfiltered = pd.read_csv("data\motionStrike_TVcodes_data.tsv", delimiter = '\t')


,CaseReferenceNumber,MotionID,CaseLocation,CaseMajorCode,CaseMinorCode,CaseTrialListType,CaseMarkingCode,CaseDispositionJudgeJurisNo,CaseDispositionDocketLegendCode,MotionResultCode,...,MotionJurisNumber,MotionDocumentPriorityCode,CaseAttorneyJuris,CaseAttorneyType,MotionTimeDuration,MotionDocumentTypeName,SelfRepBeforeMotionFileCount,SelfRepBeforeMotionDecidedCount,SelfRepBeforeMotionFileBool,SelfRepBeforeMotionDecidedBool
0,207,1751,FBT,V,1,9,16.0,416586.0,JDGDACT,GR,...,418027.0,30,418027.0,J,78.0,MOTION TO STRIKE,1,1,True,True
1,1472,10915,FBT,V,1,9,16.0,NaN,WDACT,DN,...,416586.0,30,416586.0,R,22.0,MOTION TO STRIKE,0,0,False,False
2,1494,11050,NNI,V,1,9,16.0,NaN,WDACT,GR,...,422392.0,30,422392.0,J,285.0,MOTION TO STRIKE,0,0,False,False
3,371,2708,LLI,T,90,-1,NaN,418028.0,SJD,OR,...,418028.0,30,418028.0,R,120.0,MOTION TO STRIKE,0,0,False,False
4,1422,10373,HHD,T,2,-1,NaN,422394.0,SJ,GR,...,81181.0,30,81181.0,R,153.0,MOTION TO STRIKE,0,0,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13798,4137836,26395795,HHD,T,20,-1,NaN,NaN,NaN,OR,...,415596.0,30,415596.0,R,47.0,MOTION TO STRIKE,0,0,False,False
13799,4143717,26397468,HHB,T,61,-1,NaN,NaN,NaN,DN,...,431662.0,30,431662.0,J,49.0,MOTION TO STRIKE,0,0,False,False
13800,4111125,26482434,TTD,T,29,-1,NaN,NaN,NaN,GR,...,436946.0,30,436946.0,J,47.0,MOTION TO STRIKE,1,2,True,True
13801,4026253,26397700,FBT,T,90,-1,NaN,NaN,NaN,GR,...,438581.0,30,438581.0,J,47.0,MOTION TO STRIKE,0,0,False,False


In [59]:
#important columns identifiers 
target= tab_data_unfiltered["MotionResultCode"]
#potential features
features = tab_data_unfiltered[["CaseMajorCode", "MotionTimeDuration", "CaseLocation", "CaseAttorneyJuris"]]
features

,CaseMajorCode,MotionTimeDuration,CaseLocation,CaseAttorneyJuris
0,V,78.0,FBT,418027.0
1,V,22.0,FBT,416586.0
2,V,285.0,NNI,422392.0
3,T,120.0,LLI,418028.0
4,T,153.0,HHD,81181.0
...,...,...,...,...
13798,T,47.0,HHD,415596.0
13799,T,49.0,HHB,431662.0
13800,T,47.0,TTD,436946.0
13801,T,47.0,FBT,438581.0


In [60]:
#for case location find each value like FBT NNI assign it to a key and then replace the key 
caseMajor = {}
for i in features["CaseLocation"]:
    caseMajor[i] = caseMajor.get(i, 0) + 1

Caselength = len(caseMajor)
print(f"The Length is {Caselength} and the frequencies are:\n {caseMajor}")

caseMajor = {}
for i in features["CaseMajorCode"]:
    caseMajor[i] = caseMajor.get(i, 0) + 1

Caselength = len(caseMajor)
print(f"The Length is {Caselength} and the frequencies are:\n {caseMajor}")

#assign values 1 to 15


#binary encoding to CaseMajorCode 
be = ce.BinaryEncoder(cols = 'CaseMajorCode',return_df = True)
major_encoded = be.fit_transform(features)
#not sure if this is what was intended but role with it for now
print(major_encoded)

#for CaseLocation two methods
#label encoding or frequency encoding
#less variability with near by number allocation with frequency encoding try that
fe =  ce.CountEncoder(cols = 'CaseLocation', normalize = True)
loc_encoded = fe.fit_transform(features)



#adding attorney specialization part
def calculate_attorney_specialization(specialtab):
    # Make a copy of the input dataframe to avoid the warning
    df = specialtab.copy()
    
    # Group by attorney and count case types
    case_counts = pd.crosstab(df['CaseAttorneyJuris'], 
                             df['CaseMajorCode'])
    
    # Calculate entropy with Dirichlet smoothing for each attorney
    attorney_entropy = {}
    for attorney in case_counts.index:
        counts = case_counts.loc[attorney]
        
        # Apply Dirichlet smoothing (adding 1 to each count)
        smoothed_counts = counts + 1
        # Calculate proportions
        proportions = smoothed_counts / smoothed_counts.sum()
        # Calculate entropy
        attorney_entropy[attorney] = entropy(proportions, base=2)
    
    # Add entropy back to dataframe
    df['AttorneySpecialization'] = df['CaseAttorneyJuris'].map(attorney_entropy)
    
    return df
specialentropy = calculate_attorney_specialization(tab_data_unfiltered)

# Look at entropy scores
print("\nEntropy scores:")
print(specialentropy.groupby('CaseAttorneyJuris')['AttorneySpecialization'].head())

print("\nMost specialized attorneys (lowest entropy):")
print(specialentropy.groupby('CaseAttorneyJuris')
      .agg({'AttorneySpecialization': 'first', 'CaseMajorCode': lambda x: x.value_counts().to_dict()})
      .sort_values('AttorneySpecialization')
      .head())



The Length is 15 and the frequencies are:
 {'FBT': 2003, 'NNI': 213, 'LLI': 287, 'HHD': 3094, 'FST': 1494, 'MMX': 519, 'NNH': 1528, 'KNO': 67, 'HHB': 999, 'KNL': 833, 'DBD': 489, 'UWY': 1369, 'AAN': 518, 'TTD': 183, 'WWM': 207}
The Length is 2 and the frequencies are:
 {'V': 5538, 'T': 8265}
       CaseMajorCode_0  CaseMajorCode_1  MotionTimeDuration CaseLocation  \
0                    0                1                78.0          FBT   
1                    0                1                22.0          FBT   
2                    0                1               285.0          NNI   
3                    1                0               120.0          LLI   
4                    1                0               153.0          HHD   
...                ...              ...                 ...          ...   
13798                1                0                47.0          HHD   
13799                1                0                49.0          HHB   
13800                1 

In [65]:
major_encoded

features_new = pd.concat([major_encoded['CaseMajorCode_0'], major_encoded['CaseMajorCode_1'], loc_encoded['CaseLocation'],specialentropy['AttorneySpecialization'],features['CaseAttorneyJuris'],features['MotionTimeDuration']],axis = 1 )

In [66]:
features_new

,CaseMajorCode_0,CaseMajorCode_1,CaseLocation,AttorneySpecialization,CaseAttorneyJuris,MotionTimeDuration
0,0,1,0.145113,0.999273,418027.0,78.0
1,0,1,0.145113,0.992774,416586.0,22.0
2,0,1,0.015431,0.999571,422392.0,285.0
3,1,0,0.020793,0.902393,418028.0,120.0
4,1,0,0.224154,0.977001,81181.0,153.0
...,...,...,...,...,...,...
13798,1,0,0.224154,0.980867,415596.0,47.0
13799,1,0,0.072376,0.884115,431662.0,49.0
13800,1,0,0.013258,0.991076,436946.0,47.0
13801,1,0,0.145113,0.989588,438581.0,47.0


In [ ]:
#entropy filtering for each column


,CaseMajorCode,MotionTimeDuration,CaseLocation
0,V,78.0,FBT
1,V,22.0,FBT
2,V,285.0,NNI
3,T,120.0,LLI
4,T,153.0,HHD
